In [69]:
import pandas as pd
import numpy as np
import importlib
import re
import med_utils
importlib.reload(med_utils)

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
AIBL_df = pd.read_excel('../raw_datasets/AIBL/AIBL_2023.xlsx')

In [6]:
AIBL_df_clean = med_utils.clean_dataframe_advanced(AIBL_df, missing_values=[" "], threshold=1, drop_rows_threshold=None, verbose=True)
AIBL_df_clean['Demographic.YearMonthOfBirth']=AIBL_df_clean['Demographic.YearMonthOfBirth'].astype(str) + '01'
AIBL_df_clean['Demographic.YearMonthOfBirth'] = pd.to_datetime(AIBL_df_clean['Demographic.YearMonthOfBirth'], format='%Y%m%d')
AIBL_df_clean[['Demographic.YearMonthOfBirth']] = AIBL_df_clean.groupby('AIBL ID')[['Demographic.YearMonthOfBirth']].transform(lambda x: x.ffill().bfill())
AIBL_df_clean['Progress Summary.Date of NP assessment'] = pd.to_datetime(AIBL_df_clean['Progress Summary.Date of NP assessment'], format='%d/%m/%Y')
AIBL_df_clean['Age'] = round((AIBL_df_clean['Progress Summary.Date of NP assessment'] - AIBL_df_clean['Demographic.YearMonthOfBirth']).dt.days / 365, 2)
print("Missing AIBL Age records:", AIBL_df_clean["Age"].isna().sum())
AIBL_df_clean = med_utils.fill_missing_ages_AIBL(AIBL_df_clean)
print("Missing AIBL Age records after fill given assumption of 18 month of AIBL visits:", AIBL_df_clean["Age"].isna().sum())

Original shape: (37818, 1400)
Missing values replaced: [' ']
Columns dropped: 0 ([])
Final shape: (37818, 1400)
Missing AIBL Age records: 27659
Missing AIBL Age records after fill given assumption of 18 month of AIBL visits: 3619


In [7]:
# drop rows with AIBL Age or Neuropsych.Simple Classification (status) missing
mask_missing = AIBL_df_clean["Age"].isna() | AIBL_df_clean["Neuropsych.Simple Classification"].isna()
print(f"Rows with missing Age or status to drop: {mask_missing.sum()}")
AIBL_df_clean = AIBL_df_clean.loc[~mask_missing].copy()
print(f"After dropping missings: {len(AIBL_df_clean)}")


Rows with missing Age or status to drop: 13220
After dropping missings: 24598


In [8]:
medication_columns = [col for col in AIBL_df_clean.columns if col.startswith("Medical History.Name of medication")]
AIBL_df_selected = med_utils.rename_and_select_columns(AIBL_df_clean, "../raw_datasets/naming_convension.xlsx", "AIBL", "New_Name", extra_cols=medication_columns)

In [9]:
print("The dimension of the selected AIBL dataframe is:", AIBL_df_selected.shape, "with", AIBL_df_selected['id'].nunique(), "unique participants.")
AIBL_df_selected = AIBL_df_selected[AIBL_df_selected['status'].isin(['HC', 'MCI', 'AD'])]
AIBL_df_selected['sex'] = AIBL_df_selected['sex'].map({"Male":0 , 'Female':1})
AIBL_df_selected['APOE4'] = AIBL_df_selected['APOE4'].apply(lambda x: 1 if isinstance(x, str) and 'E4' in x else 0)

print("The dimension of the selected AIBL dataframe after drop unrelated with AD is:", AIBL_df_selected.shape, "with", AIBL_df_selected['id'].nunique(), "unique participants.")
AIBL_df_selected = med_utils.clean_status_and_demos(AIBL_df_selected)
AIBL_df_selected = AIBL_df_selected.dropna(subset=['status'])
AIBL_df_selected = AIBL_df_selected[~(AIBL_df_selected['MMSE'].isna() & AIBL_df_selected['CDR'].isna())]
AIBL_df_selected = AIBL_df_selected.reset_index(drop=True)
print("The dimension of the selected AIBL dataframe is:", AIBL_df_selected.shape, "with", AIBL_df_selected['id'].nunique(), "unique participants.")
med_df=AIBL_df_selected[medication_columns]
selected_columns = AIBL_df_selected.columns.tolist()

The dimension of the selected AIBL dataframe is: (24598, 31) with 2790 unique participants.
The dimension of the selected AIBL dataframe after drop unrelated with AD is: (9503, 31) with 2503 unique participants.
Filled 0 missing status records; columns forward/back filled: ['sex', 'edu', 'APOE4']
The dimension of the selected AIBL dataframe is: (9487, 31) with 2502 unique participants.


### angiotensin-converting enzyme inhibitors (ACEi)

In [10]:
acei_files = pd.read_excel('../raw_datasets/names.xlsx', sheet_name='ACEi')
acei_names = list(set(acei_files['Drug_name'].dropna()).union(acei_files['Brand_name'].dropna()))

In [11]:
acei_matched_values = med_utils.get_matched_values(med_df, acei_names)

In [12]:
acei_matched_values

['RAMIPRIL',
 'Perindopril/Indapamide',
 'Enalapril Acetec',
 'MONOPRIEL',
 'Lisiuopril',
 'rammipril',
 'Captopril Tablets',
 'CAPOTEN',
 'Tritace (Ramipril)',
 'Enalapril',
 'Quinapril',
 'Rampipril',
 'Bace',
 'RamiprilWinthrop',
 'PERINDOPRIL',
 'monopril',
 'Monopril',
 'Indapam/perindopril',
 'Accupril',
 'Filpril (Quinapril)',
 'perendopril',
 'perindopril erbumine',
 'Fe',
 'Perindapril',
 'QUINAPRIL',
 'Lisinopril',
 'Rampril',
 'Perendopril',
 'perindopril/indapamide',
 'Azep',
 'accupril',
 '30 Perindopril',
 'Perindo',
 'MONOPRIL',
 'ACCUPRIL',
 'perindopril / indapamide',
 'rampril',
 'Fosinopril',
 'Tryzan Ramipril',
 'ramipril',
 'Enalapril Maleate',
 'ENALAPRIL',
 'Ramipril sz',
 'zestril',
 'Coversyl Plus (Perindopril, indapamde-hemhydrate',
 'perindo',
 'Ramipril',
 'PRINIVIL',
 'LISINOPRIL/LISODOR',
 'Perindopril',
 'Capoten',
 'ACE',
 'lisimopril',
 'Perindopril arginine',
 'Coversyl, perindopril, arginine',
 'perindopril',
 'Zestril',
 'Perindopril Erbumine',
 'Gop

In [13]:
values_to_drop_aceinames = {'Azep', 'Fe', 'Bace', }
acei_matched_values = [x for x in acei_matched_values if x not in values_to_drop_aceinames]

In [14]:
acei_all = med_utils.medication_all_timepoints(
    AIBL_df_selected, acei_matched_values, medication_columns, selected_columns,
    id_col="id", visit_col="visit_no"
)
# acei_all.to_csv("../preprocessed_data/AIBL/acei_all_results_AIBL_missing.csv",index=False)

### angiotensin receptor blockers (ARBs)

In [15]:
arb_files = pd.read_excel('../raw_datasets/names.xlsx', sheet_name='ARB')
arb_names = list(set(arb_files['Drug_name'].dropna()).union(arb_files['Brand_name'].dropna()))

In [16]:
arb_matched_values = med_utils.get_matched_values(med_df, arb_names)

In [17]:
arb_matched_values

['Micardis plus 80/12.5',
 'attacand',
 'Avapro',
 'Valsartan amlodipine',
 'candestartan',
 'avapro-hct',
 'Olmetec',
 'Micardis Plus 80/12.5',
 'micardis tablet',
 'candesartan',
 'AVAPRO/HCT',
 'valsartan/hydrochlorothiazide',
 'Avapro (Blood pressure)',
 'Avapro HCT   Irbesartan',
 'Telmisartan HCTZ 40/12.5',
 'Avapro HCT',
 'Teveten',
 'MICARDIS Tab',
 'Actacand',
 'AVAPRO',
 'Ibestartan',
 'Telmisartan/Hydrochlorothiazide',
 'Candesartan cilexetil',
 'micardis',
 'olmesarton',
 'atacand +',
 'Apo-telmisartan',
 'co-diovan',
 'Atacand',
 'Atacand plus',
 'micardis plus',
 'ATACAND',
 'olmesartan',
 'Irbesarton',
 'co diovan',
 'Candezartan',
 'avarpro',
 'OLMETEC',
 'Olmesartan/HCT',
 'avapro hct',
 'atacand',
 'Valsartan hydrochlorothiazide',
 'ATACAN',
 'CO-DIOVAN',
 'Irbesarten',
 'irbesartan HCT2',
 'Olmesartan',
 'antacand',
 'Irbesartan HCT',
 'Valsartan/Amlodipine',
 'Telmisartan/HCT sandoz',
 'Avapro150',
 'OLMESARTON',
 'Avapro HCT (150/12.5)',
 'Olmetec (olmesaran medoxo

In [18]:
values_to_drop_arbnames = {'VE'}
arb_matched_values = [x for x in arb_matched_values if x not in values_to_drop_arbnames]

In [19]:
arb_all = med_utils.medication_all_timepoints(
    AIBL_df_selected, arb_matched_values, medication_columns, selected_columns,
    id_col="id", visit_col="visit_no"
)
# arb_all.to_csv("../preprocessed_data/AIBL/arb_all_results_AIBL_missing.csv",index=False)

### Beta Blocker

In [20]:
beta_blockers_files = pd.read_excel('../raw_datasets/names.xlsx', sheet_name='beta_blockers')
beta_blockers_names = list(set(beta_blockers_files['Drug_name'].dropna()).union(beta_blockers_files['Brand_name'].dropna()))

In [21]:
beta_blockers_matched_values = med_utils.get_matched_values(med_df, beta_blockers_names)

In [22]:
beta_blockers_matched_values

['Timolol eye-drops 0.5%',
 'METROPOLOL',
 'Timolol Eye drops',
 'Metropolol',
 'DILATREND (Carvedilol)',
 'timolol',
 'metropolol',
 'propranolol',
 'Betaloc (Metoprolol tartrate)',
 'Propanolol',
 'Timolol',
 'toprol-XL',
 'toprol',
 'Nebivolol',
 'sotalol',
 'metroprolol',
 'bisoprodol',
 'Tenormin',
 'ATENOLOL',
 'Metopropol',
 'Carvedilol',
 'Apo-Metoprolol',
 'metaprolol',
 'Toprol XL',
 'propanolol',
 'Xalacom latanoprost timolol',
 'VE',
 'METOPROLOL (Minax)',
 'Atenolol',
 'metoprolol  AN',
 'Metoprolol Tartrate',
 'Atenol',
 'Bisoprolol',
 'nebivolol',
 'Meteprolol',
 'metopolol',
 'Bisoprolol Fumarate',
 'Sotalol',
 'Metoprolo',
 'Sotalol h/chloride',
 'atenolol',
 'SOTALOL',
 'carvediol',
 'INDERAL',
 'Betaloc (Beta blocker)',
 'Inderal',
 'Metoprolol',
 'tenormin',
 'metoprolol',
 'METROPROLOL',
 'propraholol',
 'atenalol',
 'Metoprolol tartrate (sz)',
 'carvedilol',
 'METOPROLOL',
 'ATENALOL',
 'Meloprolol',
 'Sotalol hydrocholride',
 'Tenorimin',
 'METOPOLOL',
 'Apo meto

In [23]:
values_to_drop_BetaBlknames = {'ACE', 'VE'}
beta_blockers_matched_values = [x for x in beta_blockers_matched_values if x not in values_to_drop_BetaBlknames]

In [24]:
beta_blockers_all = med_utils.medication_all_timepoints(
    AIBL_df_selected, beta_blockers_matched_values, medication_columns, selected_columns,
    id_col="id", visit_col="visit_no"
)
# beta_blockers_all.to_csv("../preprocessed_data/AIBL/beta_blockers_all_results_AIBL_missing.csv",index=False)

### Calcium Channel Blockers

In [25]:
Ca_channel_blockers_files = pd.read_excel('../raw_datasets/names.xlsx', sheet_name='Ca_channel_blockers')
Ca_channel_blockers_names = list(set(Ca_channel_blockers_files['Drug_name'].dropna()).union(Ca_channel_blockers_files['Brand_name'].dropna()))

In [26]:
Ca_channel_blockers_matched_values = med_utils.get_matched_values(med_df, Ca_channel_blockers_names)

In [27]:
Ca_channel_blockers_matched_values

['Plendil',
 'Valsartan amlodipine',
 'zanidip',
 'Felodipine ER',
 'Plendil ER',
 'ADALAT',
 'lerconidipine',
 'almodipine',
 'Diltiazem',
 'Tazac',
 'Lercanidipine',
 'AMLODIPINE',
 'felodipine',
 'Zandip',
 'Nifedipine Adefin XL',
 'LERCAN',
 'adalat',
 'Nifedipine',
 'PLENDIL ER',
 'calcium',
 'Norvac',
 'Felodipine',
 'Felodepine',
 'Verapamil',
 'ZANIDIP/ZIRCOL',
 'amlodopine',
 'CALCIUM',
 'nitedipine',
 'apo- lercanidipine',
 'Apo-amlodipine',
 'Lercanidipine Hydrochloride',
 'lercandipine',
 'NORVASC',
 'Novasc',
 'CARDIZEM',
 'Valsartan/Amlodipine',
 'PLENDIL',
 'Amrodipine',
 'amlodipine Hi Prex',
 'Pritor/Amlodipine',
 'Norvascc',
 'plendiler',
 'Zanidip, 20 ml',
 'Amlodipine Besylate',
 'Lercanidipine (Zircol)',
 'lercanidipone',
 'Amlodipine',
 'cardizem',
 'AMLODOPINE',
 'VE',
 'amlodipine',
 'diltiazem',
 'AMLO',
 'Cardizem',
 'calcium +',
 'nifedipine',
 'Amlodipine/Atorvastatin',
 'FELODIPINE',
 'Adalat',
 'Amiodipine',
 'verapamil',
 'lercan',
 'Lercanidipine-HCI',
 

In [28]:
values_to_drop_ccbnames = {'calcium +', 'Calcium', 'calcium', 'CALCIUM', 'Fe', 'VE', 'NAC',}
Ca_channel_blockers_matched_values = [x for x in Ca_channel_blockers_matched_values if x not in values_to_drop_ccbnames]

In [29]:
Ca_channel_blockers_all = med_utils.medication_all_timepoints(
    AIBL_df_selected, Ca_channel_blockers_matched_values, medication_columns, selected_columns,
    id_col="id", visit_col="visit_no"
)
# Ca_channel_blockers_all.to_csv("../preprocessed_data/AIBL/Ca_channel_blockers_all_results_AIBL_missing.csv",index=False)

### Diuretics

In [30]:
diuretics_files = pd.read_excel('../raw_datasets/names.xlsx', sheet_name='diuretics')
diuretics_names = list(set(diuretics_files['Drug_name'].dropna()).union(diuretics_files['Brand_name'].dropna()))

In [31]:
diuretics_matched_values = med_utils.get_matched_values(med_df, diuretics_names)

In [32]:
diuretics_matched_values

['Perindopril/Indapamide',
 'Spironolactone',
 'Hygroton',
 'diuretic',
 'indapamide',
 'Urex (Furosemide)',
 'iron',
 'Pelindopril and Indapamide',
 'IRON',
 'Furosemide',
 'hydrochlorothiazidie',
 'Lasix',
 'Aldactone',
 'lasix',
 'hydrchlorothiazide',
 'Spiractin (Aldactone)',
 'INDAPAMIDE',
 'Potassium',
 'spirolactone',
 'Valsartan hydrochlorothiazide',
 'Spiractin 25',
 'Valsartan/Hydrochlorothiazide',
 'Natrilix (Indapamide hemihydrate)',
 'aldactone',
 'LASIX',
 'LASIX-M',
 'perindopril/indapamide',
 'valsartan/hydrochlorothiazide',
 'furosemide',
 'lasix M',
 'Iron',
 'Spiractin (Spironolactose)',
 'perindopril / indapamide',
 'Spironolacton',
 'hygroton',
 'spiractin',
 'Irbesartan/hydrochlorothiazide',
 'indapanmide',
 'Diuretic',
 'Hydrochlorothiazide',
 'spirano lactone',
 'Indapamide SR',
 'pironolactone',
 'ACE',
 'Spiractin',
 'Telmisartan/Hydrochlorothiazide',
 'SPIRONOLACTONE',
 'Indapamide',
 'Avapro HCT  Hydrochlorothiazide',
 'hydrochlorothiazide',
 'Chlorthal',
 '

In [33]:
values_to_drop_diureticsnames = {'Iron', 'Potassium', 'ACE', 'none', 'IRON', 'iron'}
diuretics_matched_values = [x for x in diuretics_matched_values if x not in values_to_drop_diureticsnames]

In [34]:
diuretics_all = med_utils.medication_all_timepoints(
    AIBL_df_selected, diuretics_matched_values, medication_columns, selected_columns,
    id_col="id", visit_col="visit_no"
)
# diuretics_all.to_csv("../preprocessed_data/AIBL/diuretics_all_results_AIBL_missing.csv",index=False)

### statins 

In [35]:
statin_files = pd.read_excel('../raw_datasets/names.xlsx', sheet_name='statin')
statin_names = list(set(statin_files['Drug_name'].dropna()).union(statin_files['Brand_name'].dropna()))

In [36]:
statin_matched_values = med_utils.get_matched_values(med_df, statin_names)

In [37]:
statin_matched_values

['Statin',
 'Lipitor Atorvastatin',
 'Prava statin',
 'atoruastatin',
 'rosuvastatin',
 'atorvastin',
 'SIMVASTATIN',
 'atorvostatin',
 'apo rosuvastatin',
 'Atorvastatin (sz)',
 'atrovastatin',
 'atoravastatin',
 'Rovastatin',
 'Coversyl, rosuvastatin-RBX',
 'statin',
 'rosuvstatin',
 'Rosuvastatin (APO)',
 'PRAVASTATIN',
 'Crestor',
 'Lipitor 20mg',
 'Rosvastatin',
 'pravastin',
 'Pravast',
 'ASORVASTATIN',
 'Simvastatin 40',
 'Simvarstatin',
 'Rosuvastation',
 'rosivastatin',
 'Fluvastatin',
 'zocor',
 'Cretor',
 'ATORVASTATIN SANDOZ',
 'Atoruastatin',
 'Pravacol',
 'LIPITOR tab',
 'Apo-simvastatin',
 'atorvastatin',
 'amalodipine vasolate/atorvastatin calcium',
 'rosuvastatin calcium',
 'Rosuvastatin',
 'Atorvastatin (lipitor)',
 'SIMVAR/ZOCOR/ZIMSTAT',
 'Apo-atorvastatin',
 'SIMVASTIN',
 'Pravastin',
 'liptor',
 'Atrovastatin',
 'Lipitor (Atorvastatin)',
 'Zocor',
 'PRAVACHOL',
 'Pravachol',
 'crestor',
 'Ezetimibel Rosuvastatin',
 'Simvastin',
 'lescol',
 'Atorvachol/Atorvastatin

In [38]:
values_to_drop_statinnames = {'Iron', 'Potassium', 'ACE', 'none', 'IRON', 'iron'}
statin_matched_values = [x for x in statin_matched_values if x not in values_to_drop_statinnames]

In [39]:
statin_all = med_utils.medication_all_timepoints(
    AIBL_df_selected, statin_matched_values, medication_columns, selected_columns,
    id_col="id", visit_col="visit_no"
)
# statin_all.to_csv("../preprocessed_data/AIBL/statin_all_results_AIBL_missing.csv",index=False)

### metformin

In [40]:
metformin_files = pd.read_excel('../raw_datasets/names.xlsx', sheet_name='metformin')
metformin_names = list(set(metformin_files['Drug_name'].dropna()).union(metformin_files['Brand_name'].dropna()))

In [41]:
metformin_matched_values = med_utils.get_matched_values(med_df, metformin_names)

In [42]:
metformin_matched_values

['Medformin',
 'Metex',
 'jardiamet',
 'Diabex XR 1000',
 'METFORMIN',
 'Metformin Hydrochloride',
 'Diabex XR',
 'Diabex',
 'DIABEX',
 'GLUCOVANCE',
 'metformin',
 'Diabex XR ER',
 'Glucophage',
 'metformin hydrochloride',
 'metex xr',
 'Jardiamet 5/1000',
 'Formet Aspen',
 'Glucovance (500/5mg)',
 'VE',
 'Metformin HCL',
 'Diaformin Xr',
 'formetaspen',
 'DIAFORMIN 500',
 'dianformin',
 'diaformin',
 'diabex',
 'diabex XD',
 'Diaformin (1gm)',
 'Glucovance',
 'Jardiamet',
 'DIABEX XR',
 'Metformin',
 'Glucohexal',
 'medformin',
 'Diaformin',
 'diaformin xr',
 'Diaformin XR',
 'DIAFORMIN']

In [43]:
values_to_drop_metforminnames = {'VE'}
metformin_matched_values = [x for x in metformin_matched_values if x not in values_to_drop_metforminnames]

In [44]:
metformin_all = med_utils.medication_all_timepoints(
    AIBL_df_selected, metformin_matched_values, medication_columns, selected_columns,
    id_col="id", visit_col="visit_no"
)
# metformin_all.to_csv("../preprocessed_data/AIBL/metformin_all_results_AIBL_missing.csv",index=False)

### control group
never have any of the 5 type of medications

In [45]:
drug_lists = [
    acei_matched_values,
    arb_matched_values,
    beta_blockers_matched_values,
    Ca_channel_blockers_matched_values,
    diuretics_matched_values,
    statin_matched_values,
    metformin_matched_values
]

all_target_drugs = set().union(*drug_lists)

In [46]:
row_has_target_drug = (
    AIBL_df_selected[medication_columns]
      .applymap(lambda x: med_utils.contains_any_medication_type(x, all_target_drugs))
      .any(axis=1)
)
ids_with_any_target_drug = AIBL_df_selected.loc[row_has_target_drug, 'id'].unique()
controls_df = AIBL_df_selected[~AIBL_df_selected['id'].isin(ids_with_any_target_drug)].copy()


In [47]:
controls_df["Has_Medication_This_Visit"] = False
controls_df = controls_df.drop(columns=[c for c in medication_columns if c in controls_df.columns])
# controls_df.to_csv("../preprocessed_data/AIBL/controls_df_AIBL_missing.csv",index=False)

### Adjust for the later modelling in R

In [48]:
datasets = {
    "ACEi": acei_all,
    "ARB": arb_all,
    "BetaBlk": beta_blockers_all,
    "CCB": Ca_channel_blockers_all,
    "Diuretic": diuretics_all,
    "Statin": statin_all,
    "Metformin": metformin_all,
    "Control": controls_df  
}

In [49]:
merged_df = med_utils.merge_medication_longitudinal(datasets, control_key="Control", extra_cols=["visit_no", "MMSE", "CDR"])

In [50]:
merged_df

,id,visit_date,status,age,sex,edu,APOE4,ACEi,ARB,BetaBlk,CCB,Diuretic,Statin,Metformin,Total_Meds,visit_no,MMSE,CDR
0,2,2006-11-03,HC,65.30,0,16.0,0,False,False,False,False,False,False,False,0,1,29.0,0.0
1,2,2008-09-16,HC,67.17,0,16.0,0,False,False,False,False,False,False,False,0,2,30.0,0.0
2,2,2010-03-24,HC,68.69,0,16.0,0,False,False,False,False,False,False,False,0,3,30.0,0.0
3,2,2011-09-16,HC,70.17,0,16.0,0,False,False,False,False,False,False,False,0,4,30.0,0.0
4,2,2013-04-11,HC,71.74,0,16.0,0,False,False,False,False,False,False,False,0,5,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9205,2999,2022-12-12,MCI,88.34,0,NaN,0,False,False,False,False,False,False,False,0,2,30.0,1.0
9206,3004,2023-03-16,MCI,70.84,0,NaN,0,False,False,False,False,False,False,False,0,2,22.0,1.0
9207,3014,2023-03-27,HC,67.61,0,NaN,0,False,False,False,False,False,False,False,0,2,30.0,0.0
9208,3029,2022-11-17,HC,70.26,0,NaN,0,False,False,False,False,False,False,False,0,2,30.0,0.0


In [51]:
print(f"Initial rows: {len(merged_df)}")
merged_clean = merged_df.dropna(subset=['status'])
merged_clean = merged_clean[~(merged_clean['MMSE'].isna() & merged_clean['CDR'].isna())]
print(f"After dropping missings: {len(merged_clean)}")
merged_clean = merged_clean[merged_clean.groupby('id')['visit_no'].transform('nunique') > 1]
print(f"After removing number of records: {len(merged_clean)}")

Initial rows: 9210
After dropping missings: 9210
After removing number of records: 8472


In [52]:
merged_clean = med_utils.clean_and_filter_participants(
    merged_clean,
    id_col='id',
    class_col ='status',
    time_col = 'visit_no',
    medication_ever_col=None,
    min_visits=2,
    if_print=True
)

Remaining participants: 1,711
Dropped participants:   0


In [53]:
### AIBL use the age informations,
merged_clean['months_since_baseline'] = (
    merged_clean.groupby('id')['age']
      .transform(lambda x: (x - x.min()) * 12)  # convert years to months
      .round()
      .astype(int)
)

In [54]:
merged_clean = med_utils.clean_longitudinal(merged_clean)

In [55]:
# merged_clean[merged_clean.duplicated(subset=['id', 'visit_no'], keep=False)].sort_values(['id','visit_no'])

In [56]:
max_tab = merged_clean.groupby('id')['Total_Meds'].max().value_counts()
print("\nDistribution of max Total_Meds per participant:\n", max_tab)



Distribution of max Total_Meds per participant:
 Total_Meds
0    883
1    488
2    239
3     87
4     13
5      1
Name: count, dtype: int64


In [57]:
merged_clean['year_since_baseline'] = (merged_clean['months_since_baseline']/ 12).round(2)

In [58]:
merged_clean.to_csv("../preprocessed_data/AIBL/AIBL_merge_full_missing.csv",index=False)

In [59]:
print(merged_clean['id'].nunique(), "unique participants in the final AIBL dataset.")

1711 unique participants in the final AIBL dataset.


In [70]:
merged_clean = pd.read_csv('../preprocessed_data/AIBL/AIBL_cdr_imputed.csv')

In [65]:
summary_df = med_utils.baseline_summary(merged_clean)
summary_df

,Measure,Value
0,Age at baseline (year),72.1 ± 7.3
1,Gender (Female),948 (55.5%)
2,Education (year),12.7 ± 3.1
3,APOE4 (YES)**,608 (35.6%)
4,Visits (Record),2.0 – 11.0
5,Follow up intervals (Month),78.1 ± 50.4
6,CU at baseline,1210 (70.8%)
7,MCI at baseline,251 (14.7%)
8,AD at baseline,247 (14.5%)
9,Average medication taken at baseline,0.4 ± 0.7


In [71]:
med_cols_names = ["ACEi", "ARB", "BetaBlk", "CCB", "Diuretic", "Statin", "Metformin"]
med_utils.baseline_age_by_med(merged_clean, med_cols_names)

,drug,user,n,mean,sd,mean_sd,p
0,ACEi,non-user,1650,71.99,7.29,71.99 (7.29),0.000945
1,ACEi,user,58,75.02,6.51,75.02 (6.51),0.000945
2,ARB,non-user,1560,71.92,7.31,71.92 (7.31),0.000748
3,ARB,user,148,73.93,6.78,73.93 (6.78),0.000748
4,BetaBlk,non-user,1664,72.03,7.28,72.03 (7.28),0.033716
5,BetaBlk,user,44,74.50,7.40,74.5 (7.4),0.033716
6,CCB,non-user,1616,71.98,7.23,71.98 (7.23),0.019004
7,CCB,user,92,74.04,8.11,74.04 (8.11),0.019004
8,Diuretic,non-user,1678,72.03,7.28,72.03 (7.28),0.011162
9,Diuretic,user,30,75.62,7.20,75.62 (7.2),0.011162
